In [1]:
import pandas as pd
import numpy as np
from prophet import Prophet

In [2]:
data = pd.read_csv('/Users/harshitgupta/Desktop/vs /VS-Data-Den/rm_dataset_filtered.csv')

In [3]:
data['date_field'] = pd.to_datetime(data['date_field'])



In [4]:
data = data.set_index('date_field')


In [7]:
def fit_model(group):
    # Fit Prophet model
    model = Prophet(weekly_seasonality=True)
    group_df = group.reset_index().rename(columns={'sales': 'y'})
    group_df = group_df.rename(columns={'date_field': 'ds'})
    model.fit(group_df)
    future = model.make_future_dataframe(periods=52)
    forecast = model.predict(future)
    return forecast['yhat'].values[-52:]


In [8]:
forecasts = []
for rm, group in data.groupby('regional_master')['sales']:
    forecast_data = fit_model(group)
    forecast_df = pd.DataFrame({'regional_master': rm, 'forecast': forecast_data})
    forecasts.append(forecast_df)


18:30:02 - cmdstanpy - INFO - Chain [1] start processing
18:30:02 - cmdstanpy - INFO - Chain [1] done processing
18:30:02 - cmdstanpy - INFO - Chain [1] start processing
18:30:02 - cmdstanpy - INFO - Chain [1] done processing
18:30:03 - cmdstanpy - INFO - Chain [1] start processing
18:30:03 - cmdstanpy - INFO - Chain [1] done processing
18:30:03 - cmdstanpy - INFO - Chain [1] start processing
18:30:03 - cmdstanpy - INFO - Chain [1] done processing
18:30:03 - cmdstanpy - INFO - Chain [1] start processing
18:30:03 - cmdstanpy - INFO - Chain [1] done processing
18:30:04 - cmdstanpy - INFO - Chain [1] start processing
18:30:04 - cmdstanpy - INFO - Chain [1] done processing
18:30:04 - cmdstanpy - INFO - Chain [1] start processing
18:30:04 - cmdstanpy - INFO - Chain [1] done processing
18:30:04 - cmdstanpy - INFO - Chain [1] start processing
18:30:04 - cmdstanpy - INFO - Chain [1] done processing
18:30:04 - cmdstanpy - INFO - Chain [1] start processing
18:30:05 - cmdstanpy - INFO - Chain [1]

In [9]:
forecasts = pd.concat(forecasts, ignore_index=True)

In [19]:
def smape(actual, forecast):
    """
    Calculate the Symmetric Mean Absolute Percentage Error (SMAPE)
    """
    n = len(actual)
    combined = np.abs(actual) + np.abs(forecast)
    smape_values = np.abs(actual - forecast) / combined * 200 / n
    return np.mean(smape_values)


In [20]:
smape_scores = []
for rm, group in forecasts.groupby('regional_master'):
    actual = data.loc[data['regional_master'] == rm, 'sales'].values[-52:]
    forecast_values = group['forecast'].values
    smape_score = smape(actual, forecast_values)
    smape_scores.append(smape_score)

In [21]:
smape_scores

[0.5140206139690241,
 3.3639326855289675,
 3.846153846153846,
 0.9116773637201833,
 1.7330735238460073,
 2.729910650443588,
 1.9785258323378818,
 0.5349221419093129,
 3.846153846153846,
 0.8088113473911798,
 2.0630025325196386,
 0.24003215868087754,
 0.4789850488272655,
 0.2982267514915632,
 0.8434716104936258,
 0.7152372768165793,
 1.8749760671849565,
 2.620606948656811,
 3.846153846153846,
 1.1064922412909086,
 0.43008821425025656,
 0.5691155466011701,
 0.594203612892898,
 2.8955449867352696,
 3.6099532962173053,
 1.3387497194008018,
 0.6856295175973699,
 0.5148360179319221,
 0.3939971043032469,
 0.32904502408357883,
 0.6920818513635905,
 1.0597769046136525,
 0.8135467697194151,
 3.037211543112823,
 0.8059801926165137,
 0.621602120137267,
 3.846153846153846,
 0.3469019631874956,
 2.518155254792237,
 3.4846575993678335,
 0.8167890408645877,
 0.36570107141272934,
 3.7589948741509867,
 0.7086475290685398,
 0.3931154787011795,
 3.7785496900308475,
 3.616340326178272,
 1.9889437412089057,

In [22]:
mean_smape = np.mean(smape_scores)

In [23]:
mean_smape

1.5846126866941623

In [24]:
print(f"Mean SMAPE across all regional_masters: {mean_smape:.4f}")


Mean SMAPE across all regional_masters: 1.5846
